# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 4 2023 2:45PM,254269,10,8984110,Eywa Pharma Inc.,Released
1,Jan 4 2023 2:45PM,254269,10,8984111,Eywa Pharma Inc.,Released
2,Jan 4 2023 2:45PM,254269,10,8984112,Eywa Pharma Inc.,Released
3,Jan 4 2023 2:45PM,254269,10,8984113,Eywa Pharma Inc.,Released
4,Jan 4 2023 2:45PM,254269,10,8984115,Eywa Pharma Inc.,Released
5,Jan 4 2023 2:45PM,254269,10,8984117,Eywa Pharma Inc.,Released
6,Jan 4 2023 2:45PM,254269,10,8984671,Eywa Pharma Inc.,Released
7,Jan 4 2023 2:45PM,254269,10,8985225,Eywa Pharma Inc.,Released
8,Jan 4 2023 2:45PM,254269,10,8985226,Eywa Pharma Inc.,Released
9,Jan 4 2023 2:45PM,254269,10,8985858,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
18,254290,Released,1
19,254291,Released,1
20,254293,Released,1
21,254294,Released,1
22,254295,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254290,NaN,1.0,1.0
254291,NaN,NaN,1.0
254293,NaN,NaN,1.0
254294,NaN,NaN,1.0
254295,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254196,0.0,0.0,17.0
254206,1.0,0.0,16.0
254225,0.0,1.0,0.0
254230,36.0,5.0,58.0
254249,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254196,0,0,17
254206,1,0,16
254225,0,1,0
254230,36,5,58
254249,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254196,0,0,17
1,254206,1,0,16
2,254225,0,1,0
3,254230,36,5,58
4,254249,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254196,,,17
1,254206,1,,16
2,254225,,1,
3,254230,36,5,58
4,254249,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 4 2023 2:45PM,254269,10,Eywa Pharma Inc.
11,Jan 4 2023 2:36PM,254295,10,"Virtue Rx, LLC"
13,Jan 4 2023 2:31PM,254294,16,Sartorius Bioprocess Solutions
14,Jan 4 2023 2:27PM,254293,10,"Virtue Rx, LLC"
15,Jan 4 2023 1:57PM,254291,22,"NBTY Global, Inc."
16,Jan 4 2023 1:56PM,254290,10,Methapharm-G
17,Jan 4 2023 1:56PM,254290,10,"Methapharm, Inc."
18,Jan 4 2023 1:56PM,254289,10,"Nextsource Biotechnology, LLC"
19,Jan 4 2023 12:47PM,254282,19,"AdvaGen Pharma, Ltd"
33,Jan 4 2023 12:39PM,254278,16,Innova Softgel LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 4 2023 2:45PM,254269,10,Eywa Pharma Inc.,,,11
1,Jan 4 2023 2:36PM,254295,10,"Virtue Rx, LLC",,,2
2,Jan 4 2023 2:31PM,254294,16,Sartorius Bioprocess Solutions,,,1
3,Jan 4 2023 2:27PM,254293,10,"Virtue Rx, LLC",,,1
4,Jan 4 2023 1:57PM,254291,22,"NBTY Global, Inc.",,,1
5,Jan 4 2023 1:56PM,254290,10,Methapharm-G,,1,1
6,Jan 4 2023 1:56PM,254290,10,"Methapharm, Inc.",,1,1
7,Jan 4 2023 1:56PM,254289,10,"Nextsource Biotechnology, LLC",,,1
8,Jan 4 2023 12:47PM,254282,19,"AdvaGen Pharma, Ltd",,,14
9,Jan 4 2023 12:39PM,254278,16,Innova Softgel LLC,,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 4 2023 2:45PM,254269,10,Eywa Pharma Inc.,11,,
1,Jan 4 2023 2:36PM,254295,10,"Virtue Rx, LLC",2,,
2,Jan 4 2023 2:31PM,254294,16,Sartorius Bioprocess Solutions,1,,
3,Jan 4 2023 2:27PM,254293,10,"Virtue Rx, LLC",1,,
4,Jan 4 2023 1:57PM,254291,22,"NBTY Global, Inc.",1,,
5,Jan 4 2023 1:56PM,254290,10,Methapharm-G,1,1,
6,Jan 4 2023 1:56PM,254290,10,"Methapharm, Inc.",1,1,
7,Jan 4 2023 1:56PM,254289,10,"Nextsource Biotechnology, LLC",1,,
8,Jan 4 2023 12:47PM,254282,19,"AdvaGen Pharma, Ltd",14,,
9,Jan 4 2023 12:39PM,254278,16,Innova Softgel LLC,,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 4 2023 2:45PM,254269,10,Eywa Pharma Inc.,11,,
1,Jan 4 2023 2:36PM,254295,10,"Virtue Rx, LLC",2,,
2,Jan 4 2023 2:31PM,254294,16,Sartorius Bioprocess Solutions,1,,
3,Jan 4 2023 2:27PM,254293,10,"Virtue Rx, LLC",1,,
4,Jan 4 2023 1:57PM,254291,22,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 4 2023 2:45PM,254269,10,Eywa Pharma Inc.,11.0,NaN,NaN
1,Jan 4 2023 2:36PM,254295,10,"Virtue Rx, LLC",2.0,NaN,NaN
2,Jan 4 2023 2:31PM,254294,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN
3,Jan 4 2023 2:27PM,254293,10,"Virtue Rx, LLC",1.0,NaN,NaN
4,Jan 4 2023 1:57PM,254291,22,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 4 2023 2:45PM,254269,10,Eywa Pharma Inc.,11.0,0.0,0.0
1,Jan 4 2023 2:36PM,254295,10,"Virtue Rx, LLC",2.0,0.0,0.0
2,Jan 4 2023 2:31PM,254294,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0
3,Jan 4 2023 2:27PM,254293,10,"Virtue Rx, LLC",1.0,0.0,0.0
4,Jan 4 2023 1:57PM,254291,22,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2034171,34.0,3.0,0.0
15,508500,59.0,5.0,36.0
16,762837,2.0,1.0,0.0
18,254277,1.0,0.0,0.0
19,1016968,30.0,2.0,1.0
21,254266,1.0,0.0,0.0
22,254291,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2034171,34.0,3.0,0.0
1,15,508500,59.0,5.0,36.0
2,16,762837,2.0,1.0,0.0
3,18,254277,1.0,0.0,0.0
4,19,1016968,30.0,2.0,1.0
5,21,254266,1.0,0.0,0.0
6,22,254291,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,34.0,3.0,0.0
1,15,59.0,5.0,36.0
2,16,2.0,1.0,0.0
3,18,1.0,0.0,0.0
4,19,30.0,2.0,1.0
5,21,1.0,0.0,0.0
6,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,34.0
1,15,Released,59.0
2,16,Released,2.0
3,18,Released,1.0
4,19,Released,30.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,21,22
Status,,,,,,,
Completed,0.0,36.0,0.0,0.0,1.0,0.0,0.0
Executing,3.0,5.0,1.0,0.0,2.0,0.0,0.0
Released,34.0,59.0,2.0,1.0,30.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,21,22
0,Completed,0.0,36.0,0.0,0.0,1.0,0.0,0.0
1,Executing,3.0,5.0,1.0,0.0,2.0,0.0,0.0
2,Released,34.0,59.0,2.0,1.0,30.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,21,22
0,Completed,0.0,36.0,0.0,0.0,1.0,0.0,0.0
1,Executing,3.0,5.0,1.0,0.0,2.0,0.0,0.0
2,Released,34.0,59.0,2.0,1.0,30.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()